In [3]:
#Setting up Java, Apache Spark and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://www.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar -zxf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

--2019-11-11 02:57:03--  https://www.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
Resolving www.apache.org (www.apache.org)... 95.216.24.32, 40.79.78.1, 2a01:4f9:2a:185f::2
Connecting to www.apache.org (www.apache.org)|95.216.24.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230091034 (219M) [application/x-gzip]
Saving to: ‘spark-2.4.4-bin-hadoop2.7.tgz’

spark-2.4.4-bin-had 100%[===================>] 219.43M  44.1MB/s    in 6.9s    

2019-11-11 02:57:10 (31.8 MB/s) - ‘spark-2.4.4-bin-hadoop2.7.tgz’ saved [230091034/230091034]



In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init("/content/spark-2.4.4-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


In [6]:
import urllib.request
import pandas as pd 
import zipfile
import os
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

import os

small=urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-latest-small.zip','/content/drive/My Drive/small.zip')
big=urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-latest.zip','/content/drive/My Drive/big.zip')

with zipfile.ZipFile('/content/drive/My Drive/small.zip', "r") as z:
   z.extractall('/content/drive/My Drive/')
    
with zipfile.ZipFile('/content/drive/My Drive/big.zip', "r") as z:
   z.extractall('/content/drive/My Drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
movies= spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/ml-latest/movies.csv")
movies.take(3)


[Row(movieId='1', title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId='2', title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId='3', title='Grumpier Old Men (1995)', genres='Comedy|Romance')]

In [8]:
ratings= spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/ml-latest/ratings.csv")
ratings.take(3)   



[Row(userId='1', movieId='307', rating='3.5', timestamp='1256677221'),
 Row(userId='1', movieId='481', rating='3.5', timestamp='1256677456'),
 Row(userId='1', movieId='1091', rating='1.5', timestamp='1256677471')]

In [9]:
mergedDF = ratings.join(movies, movies.movieId == ratings.movieId,how='left') # Could also use  pd.merge(ratings,movies,on=['movieId'],how='left')
mergedDF.head(5)

[Row(userId='1', movieId='307', rating='3.5', timestamp='1256677221', movieId='307', title='Three Colors: Blue (Trois couleurs: Bleu) (1993)', genres='Drama'),
 Row(userId='1', movieId='481', rating='3.5', timestamp='1256677456', movieId='481', title='Kalifornia (1993)', genres='Drama|Thriller'),
 Row(userId='1', movieId='1091', rating='1.5', timestamp='1256677471', movieId='1091', title="Weekend at Bernie's (1989)", genres='Comedy'),
 Row(userId='1', movieId='1257', rating='4.5', timestamp='1256677460', movieId='1257', title='Better Off Dead... (1985)', genres='Comedy|Romance'),
 Row(userId='1', movieId='1449', rating='4.5', timestamp='1256677264', movieId='1449', title='Waiting for Guffman (1996)', genres='Comedy')]

In [10]:
mergedDF.count()

27753444

In [11]:
temp=mergedDF.rdd
genres=temp.map(lambda tokens: (tokens[6])).cache()
genres.take(3)


['Drama', 'Drama|Thriller', 'Comedy']

In [0]:
WordsCount=genres.flatMap(lambda line: line.split("|")).map(lambda word: (word, 1))
WordsCount.count()

In [0]:
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
DistinctWordsCount.count()

In [0]:
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
#print most frequent 20 words
SortedWordsCount.top(20)


In [0]:
SortedWordsCount.collect()
Son=SortedWordsCount.toDF(['Frequency', 'Genre']).toPandas()
Son2=Son.nlargest(20, 'Frequency', keep='first')
Son2.head()

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
Son2.plot(kind='bar',x='Genre', y='Frequency',figsize= (18, 6) )
plt.show()